In [ ]:
import cv2
import os
import time
import scipy.io
import numpy as np
from skimage import measure
from collections import deque
from google.colab import drive
import matplotlib.pyplot as plt
from skimage.segmentation import find_boundaries


# Google Drive
drive.mount('/content/drive')


## Part 1 Definition of SLIC class

In [ ]:
class SLIC:
    def __init__(self, img, step, m):
        self.img = img
        self.height, self.width = img.shape[:2]
        self._convertToLAB()
        self.step = step
        self.m = m
        self.ws = step
        self.FLT_MAX = 1000000
        self.ITERATIONS = 10

    def _convertToLAB(self):
        try:
            import cv2
            self.labimg = cv2.cvtColor(self.img, cv2.COLOR_BGR2LAB).astype(np.float64)
        except ImportError as e:
            print(f"OpenCV import failed: {e}, using custom RGB to LAB conversion.")
            self.labimg = np.copy(self.img)

            for i in range(self.labimg.shape[0]):
                for j in range(self.labimg.shape[1])
                    rgb_reversed = tuple(reversed(self.labimg[i, j]))
                    self.labimg[i, j] = self._rgb2lab(rgb_reversed)

    def _rgb2lab(self, inputColor):
        ref_X, ref_Y, ref_Z = 95.047, 100.000, 108.883
        RGB = [float(value) / 255 for value in inputColor]
        RGB = [( (value + 0.055) / 1.055 ) ** 2.4 if value > 0.04045 else value / 12.92 for value in RGB]
        RGB = [value * 100 for value in RGB]

        X = round(RGB[0] * 0.4124 + RGB[1] * 0.3576 + RGB[2] * 0.1805, 4)
        Y = round(RGB[0] * 0.2126 + RGB[1] * 0.7152 + RGB[2] * 0.0722, 4)
        Z = round(RGB[0] * 0.0193 + RGB[1] * 0.1192 + RGB[2] * 0.9505, 4)

        XYZ = [X / ref_X, Y / ref_Y, Z / ref_Z]
        XYZ = [value ** (1/3) if value > 0.008856 else (7.787 * value) + (16 / 116) for value in XYZ]

        L = round((116 * XYZ[1]) - 16, 4)
        a = round(500 * (XYZ[0] - XYZ[1]), 4)
        b = round(200 * (XYZ[1] - XYZ[2]), 4)

        return [L, a, b]

    def generateSuperPixels(self):
        self._initData()
        indnp = np.mgrid[0:self.height, 0:self.width].swapaxes(0, 2).swapaxes(0, 1)

        for i in range(self.ITERATIONS):
            self.distances = self.FLT_MAX * np.ones(self.img.shape[:2])

            for j in range(self.centers.shape[0]):
                xlow = max(int(self.centers[j][3] - self.step), 0)
                xhigh = min(int(self.centers[j][3] + self.step), self.width)
                ylow = max(int(self.centers[j][4] - self.step), 0)
                yhigh = min(int(self.centers[j][4] + self.step), self.height)

                cropimg = self.labimg[ylow:yhigh, xlow:xhigh]
                colordiff = cropimg - self.labimg[int(self.centers[j][4]), int(self.centers[j][3])]
                colorDist = np.sqrt(np.sum(np.square(colordiff), axis=2))
                yy, xx = np.ogrid[ylow:yhigh, xlow:xhigh]
                pixdist = np.sqrt((yy - self.centers[j][4])**2 + (xx - self.centers[j][3])**2)
                dist = np.sqrt((colorDist / self.m)**2 + (pixdist / self.ws)**2)

                distanceCrop = self.distances[ylow:yhigh, xlow:xhigh]
                idx = dist < distanceCrop
                distanceCrop[idx] = dist[idx]
                self.distances[ylow:yhigh, xlow:xhigh] = distanceCrop
                self.clusters[ylow:yhigh, xlow:xhigh][idx] = j

            for k in range(len(self.centers)):
                idx = (self.clusters == k)
                colornp = self.labimg[idx]
                distnp = indnp[idx]
                self.centers[k][:3] = np.sum(colornp, axis=0)
                sumy, sumx = np.sum(distnp, axis=0)
                self.centers[k][3:] = sumx, sumy
                self.centers[k] /= np.sum(idx)

            print(f'Iteration={i+1}')

        print("done")


    def _initData(self):
        self.clusters = -1 * np.ones(self.img.shape[:2])
        self.distances = self.FLT_MAX * np.ones(self.img.shape[:2])
        centers = [[*self.labimg[self._findLocalMinimum(center=(i, j))[1], self._findLocalMinimum(center=(i, j))[0]], i, j]
                  for j in range(self.step, self.height - self.step//2, self.step)
                  for i in range(self.step, self.width - self.step//2, self.step)]
        self.centers = np.array(centers)
        self.center_counts = np.zeros(len(centers))


    def createConnectivity(self):
        dx4 = [-1, 0, 1, 0]
        dy4 = [0, -1, 0, 1]
        lims = int(self.width * self.height / self.centers.shape[0])
        new_clusters = -1 * np.ones(self.img.shape[:2]).astype(np.int64)

        label = 0
        for i in range(self.width):
            for j in range(self.height):
                if new_clusters[j, i] != -1:
                    continue
                elements = deque([(j, i)])
                adjlabel = -1
                count = 1
                new_clusters[j, i] = label
                while elements:
                    y, x = elements.popleft()
                    for dx, dy in zip(dx4, dy4):
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < self.width and 0 <= ny < self.height:
                            if new_clusters[ny, nx] == -1 and self.clusters[j, i] == self.clusters[ny, nx]:
                                elements.append((ny, nx))
                                new_clusters[ny, nx] = label
                                count += 1
                            elif new_clusters[ny, nx] >= 0:
                                adjlabel = new_clusters[ny, nx]
                if count <= lims >> 2 and adjlabel != -1:
                    new_clusters[new_clusters == label] = adjlabel
                    label -= 1

                label += 1

    def displayContours(self, color):
        dx8 = [-1, -1, 0, 1, 1, 1, 0, -1]
        dy8 = [0, -1, -1, -1, 0, 1, 1, 1]

        isTaken = np.zeros(self.img.shape[:2], np.bool_)

        for i in range(self.width):
            for j in range(self.height):
                nr_p = sum(
                    0 <= i + dx < self.width and 0 <= j + dy < self.height and
                    not isTaken[j + dy, i + dx] and self.clusters[j, i] != self.clusters[j + dy, i + dx]
                    for dx, dy in zip(dx8, dy8)
                )

                if nr_p >= 2:
                    isTaken[j, i] = True

        self.img[isTaken] = color


    def _findLocalMinimum(self, center):
        min_grad = self.FLT_MAX
        loc_min = center

        for i in range(center[0] - 1, center[0] + 2):
            for j in range(center[1] - 1, center[1] + 2):
                if i + 1 < self.width and j + 1 < self.height:
                    c1 = self.labimg[j + 1, i][0]
                    c2 = self.labimg[j, i + 1][0]
                    c3 = self.labimg[j, i][0]
                    grad = abs(c1 - c3) + abs(c2 - c3)
                    if grad < min_grad:
                        min_grad = grad
                        loc_min = [i, j]

        return loc_min


    def get_segmentation(self):
        return self.clusters.copy()


## Part 2 Generation of **single** superpixel segmentation image

In [ ]:
def main():

    img_path = 'Path to your image in Google drive'
    img = cv2.imread(img_path)
    nr_superpixels = 1000
    m = 40
    step = int((img.shape[0]*img.shape[1]/nr_superpixels)**0.5)

    slic = SLIC(img, step, m)
    start_time = time.time()
    slic.generateSuperPixels()
    slic.createConnectivity()
    slic.displayContours([255, 255, 255])
    end_time = time.time()


    from matplotlib import pyplot as plt
    plt.imshow(cv2.cvtColor(slic.img, cv2.COLOR_BGR2RGB))
    plt.show()

    output_path = 'The path to your image segmentation results in Google drive'
    cv2.imwrite(output_path, slic.img)
    print("Processing time: {:.2f} seconds".format(end_time - start_time))


## Part 3 **Batch** generation of superpixel segmentation images and segmentation markers

In [ ]:
def create_subfolders(base_folder):
    image_folder = os.path.join(base_folder, "images")
    segmentation_folder = os.path.join(base_folder, "segmentations")

    if not os.path.exists(image_folder):
        os.makedirs(image_folder)

    if not os.path.exists(segmentation_folder):
        os.makedirs(segmentation_folder)

    return image_folder, segmentation_folder
def process_folder(folder_path, output_folder, nr_superpixels, m):
    image_folder, segmentation_folder = create_subfolders(output_folder)
    segmentation_results = {}
    processed_images = []
    file_names = []

    for i, file_name in enumerate(sorted(os.listdir(folder_path))):
        if file_name.endswith('.jpg'):
            img_path = os.path.join(folder_path, file_name)
            img = cv2.imread(img_path)
            step = int((img.shape[0]*img.shape[1]/nr_superpixels)**0.5)

            slic = SLIC(img, step, m)
            slic.generateSuperPixels()
            slic.createConnectivity()
            slic.displayContours([255, 255, 255])

            segmentation = slic.get_segmentation()
            segmentation_results[file_name] = segmentation

            output_image_path = os.path.join(image_folder, f"SLIC_{file_name}")
            cv2.imwrite(output_image_path, slic.img)

            segmentation_path = os.path.join(segmentation_folder, f"SLIC_segmentation_{file_name}.npy")
            np.save(segmentation_path, segmentation)

            if i < 5:
                processed_images.append(slic.img)
                file_names.append(file_name)

            print(f"Processed and saved: {file_name}")

    return processed_images, file_names, segmentation_results

def main():
    input_folder = 'Path to the image dataset to be segmented in Google drive'
    output_folder = 'Storage path in Google drive for superpixel segmentation result images and segmentation markers'
    nr_superpixels = 1000
    m = 40

    processed_images, file_names, segmentations = process_folder(input_folder, output_folder, nr_superpixels, m)

    for img, name in zip(processed_images, file_names):
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(name)
        plt.show()

## Part 4 Processing of the dataset to be used

In [ ]:
import os
import scipy.io

def extract_segmentation_labels_from_mat(file_path):
    data = scipy.io.loadmat(file_path)
    ground_truth = data.get('groundTruth')
    if ground_truth is None:
        raise ValueError("Segmentation data not found in the .mat file")

    all_segmentations = []
    for i in range(ground_truth.shape[1]):
        segmentation = ground_truth[0, i][0, 0][0]
        all_segmentations.append(segmentation)

    return all_segmentations

def process_mat_files(image_folder, mat_folder):
    segmentations = {}

    for file_name in sorted(os.listdir(image_folder)):
        if file_name.endswith('.jpg'):
            mat_file_path = os.path.join(mat_folder, file_name.replace('.jpg', '.mat'))
            if os.path.exists(mat_file_path):
                segmentation_labels = extract_segmentation_labels_from_mat(mat_file_path)
                segmentations[file_name] = segmentation_labels
            else:
                print(f"Warning: No corresponding .mat file found for {file_name}")

    return segmentations

# Path to the folder where the images and .mat files are stored
image_folder = 'Path to the folder where the images files are stored'
mat_folder = 'Path to the folder where the .mat files are stored'

all_segmentations = process_mat_files(image_folder, mat_folder)

## Part 5 Quantitative Evaluation

In [ ]:
def boundary_recall(slic_segmentation, ground_truth):
    slic_segmentation = slic_segmentation.astype(int)
    ground_truth = ground_truth.astype(int)

    slic_boundaries = find_boundaries(slic_segmentation, mode='outer')
    gt_boundaries = find_boundaries(ground_truth, mode='outer')

    matched_boundary = np.logical_and(slic_boundaries, gt_boundaries)
    recall = np.sum(matched_boundary) / np.sum(gt_boundaries)
    return recall

def undersegmentation_error(slic_segmentation, ground_truth):
    ue = 0.0
    slic_segmentation = slic_segmentation.astype(int)
    ground_truth = ground_truth.astype(int)

    for region in measure.regionprops(slic_segmentation):
        gt_region = ground_truth[region.coords[:, 0], region.coords[:, 1]]
        if len(gt_region) == 0:
            continue
        majority_label = np.bincount(gt_region).argmax()
        leakage = np.sum(gt_region != majority_label)
        ue += leakage / region.area
    return ue / len(np.unique(slic_segmentation))

br_list = []
ue_list = []

for file_name, ground_truth_segmentations in all_segmentations.items():
    slic_segmentation = np.load(f' storage path for segmentations/SLIC_segmentation_{file_name}.npy')

    ground_truth_segmentation = ground_truth_segmentations[0]

    br = boundary_recall(slic_segmentation, ground_truth_segmentation)
    ue = undersegmentation_error(slic_segmentation, ground_truth_segmentation)

    br_list.append(br)
    ue_list.append(ue)

average_br = np.mean(br_list)
average_ue = np.mean(ue_list)

print(f"Average Boundary Recall: {average_br}")
print(f"Average Undersegmentation Error: {average_ue}")


## Execute the main function

In [ ]:
if __name__ == '__main__':
    main()